### Creating corpus of interaction tweets

In [62]:
import pandas as pd
import numpy as np
import tweepy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

#### Load the members of democrat and republican lists

In [2]:
dem_df = pd.read_csv('dem.csv')
rep_df = pd.read_csv('rep.csv')

In [3]:
dem_names = dem_df.Screen_name.tolist()
rep_names = rep_df.Screen_name.tolist()

In [5]:
all_names = [name for name in dem_names]
all_names.extend(rep_names)
len(all_names)

392

In [6]:
print(len(dem_names))
print(len(rep_names))

200
192


In [7]:
dem_df.head()

,Name,Screen_name,User_Id
0,US Rep. Darren Soto,RepDarrenSoto,818713465653051392
1,Senator Jacky Rosen,SenJackyRosen,818554054309715969
2,US Rep. Al Lawson Jr,RepAlLawsonJr,818472418620608512
3,Adriano Espaillat,RepEspaillat,817076257770835968
4,Rep. Blunt Rochester,RepLBR,817050219007328258


In [8]:
rep_df.head()

,Name,Screen_name,User_Id
0,Rep. Chip Roy,RepChipRoy,1082790600292925440
1,Rep. Greg Pence,RepGregPence,1082369392229400576
2,John Joyce,RepJohnJoyce,1082311988926124036
3,Carol Miller,RepCarolMiller,1081318716573470720
4,John Rose,RepJohnRose,1081312310059253763


In [9]:
auth = tweepy.OAuthHandler('rqc3QgAjT8SMEM3L45K9Hf4em','xEo1bMUirS8Lc4CbyFjvxlU6VmjW1ouWu7s1LvLWyasbEHoXez')
auth.set_access_token('1035288698127040512-iZiV5ZhlYXLfivbxZtS98bDAdWB2eb','HnKIhSb143B9qNdxK88mFEnRjAWeBFNYNqhYRLqhPRlxh')

api = tweepy.API(auth)


### Extracting tweets of the usernames

In [10]:
tweets_dict = {
              "created_at":[],
              "id": [],
              "text": [],
              "retweets" : [],
              "retweet_count" : [],
              "user_screenname":[],
              "in_reply_to_screen_name":[],
              "in_reply_to_status_id" :[],
              "quoted_status":[],
              "hashtags":[],
              "user_mentions":[],
              "type_of_interaction":[],
                }


In [12]:
for screenname in all_names:
    try:
        stuff = api.user_timeline(screen_name = screenname,tweet_mode='extended')
        for status in stuff:
           
            if status.retweet_count>0 or status.in_reply_to_screen_name:
                tweets_dict['created_at'].append(status.created_at)
                tweets_dict['id'].append(status.id)
                tweets_dict['text'].append(status.full_text)
                tweets_dict['retweets'].append(status.retweets)
                tweets_dict['retweet_count'].append(status.retweet_count)
                tweets_dict['user_screenname'].append(status.user.screen_name)
                tweets_dict['in_reply_to_screen_name'].append(status.in_reply_to_screen_name)
                tweets_dict['in_reply_to_status_id'].append(status.in_reply_to_status_id_str)
                
                ### If it is a quoted tweet
                if status.is_quote_status:
                    try:
                        tweets_dict['quoted_status'].append(status.quoted_status)
                    except AttributeError:
                        tweets_dict['quoted_status'].append(None)
                                 
                else:
                    tweets_dict['quoted_status'].append(None)
           
        
                ### get hashtags
                hashtags = status.entities['hashtags']
                hastags_list = []
                
                for i_iter in range(0,len(hashtags)):
                    hastags_list.append(hashtags[i_iter]['text'])
                    
                tweets_dict['hashtags'].append(hastags_list)
                
                
                ### get user mentions of the tweet
                user_mentions = (status.entities['user_mentions'])
                user_mention_list = []
                
                for i_iter in range(0,len(user_mentions)):
                    user_mention_list.append(user_mentions[i_iter]['screen_name'])
                    
                tweets_dict['user_mentions'].append(user_mention_list)
                
                ### user mention interaction --> type_of_interaction
                interaction_string=""
                if status.user.screen_name in rep_names:
                    for each_user_mention in user_mention_list:
                        if (each_user_mention in dem_names) or each_user_mention=='HouseDemocrats':
                            interaction_string
                            interaction_string = "Rep->Dem"
                            break
                        elif (each_user_mention in rep_names) or each_user_mention=='HouseGOP':
                            interaction_string = "Rep->Rep"
                            break
                elif status.user.screen_name in dem_names:
                    for each_user_mention in user_mention_list:
                        if (each_user_mention in rep_names) or each_user_mention=='HouseGOP':
                            interaction_string = "Dem->Rep"
                            break
                        elif (each_user_mention in dem_names) or each_user_mention=='HouseDemocrats':
                            interaction_string = "Dem->Dem"
                            break
                
                tweets_dict['type_of_interaction'].append(interaction_string)          
                            
                    
    except tweepy.TweepError:
        print("Failed to run the command on that user, Skipping...",screenname)

    

Failed to run the command on that user, Skipping... nikiinthehouse


In [14]:
### saving it into a dataframe
tweets_dict_data = pd.DataFrame(tweets_dict)

In [15]:
len(tweets_dict_data)

7118

#### Get the name of the author and the text of the original text if the tweet extracted is a quoted tweet

In [17]:
quoted_text = []
quoted_text_screenname = []
for tw in tweets_dict_data.quoted_status:
    if tw!=None:
        quoted_text.append(tw.full_text)
        quoted_text_screenname.append(tw.user.screen_name)
    else:
        quoted_text.append(None)
        quoted_text_screenname.append(None)

In [18]:
tweets_dict_data['quoted_text_original']=quoted_text
tweets_dict_data['quoted_text_original_screenname']=quoted_text_screenname

#### Get the name of the author if the tweet extracted is a retweet 

In [19]:
orig_screenname=[]
text=[]

for tw in tweets_dict_data['text'].tolist():
    if 'RT @' in tw:
        orig_screenname.append(tw.split('@')[1].split(':')[0])
        text.append(tw.split(':')[1])
    else:
        orig_screenname.append(None)
        text.append(tw)
        

In [20]:
tweets_dict_data['retweeting_orig_screenname']=orig_screenname
tweets_dict_data['text']=text

#### Get the name of the author and the text of the original text if the tweet extracted is a reply to a tweet

In [21]:
orig_text=[]

for i in tweets_dict_data.in_reply_to_status_id:
    if i!=None:
        try:
            orig_text.append(api.get_status(i).text)
            
        except tweepy.TweepError:
            orig_text.append(None)
            
    else:
        orig_text.append(None)

In [22]:

orig_text_author = []        
        
for i in tweets_dict_data.in_reply_to_status_id:
    if i!=None:
        try:
            
            orig_text_author.append(api.get_status(i).user.screen_name)
        except tweepy.TweepError:
           
            orig_text_author.append(None)
    else:
       
        orig_text_author.append(None)

In [27]:
tweets_dict_data['orig_text_replied_to']=orig_text
tweets_dict_data['orig_author_replied_to']=orig_text_author

In [28]:
tweets_dict_data.head()

,created_at,id,text,retweets,retweet_count,user_screenname,in_reply_to_screen_name,in_reply_to_status_id,quoted_status,hashtags,user_mentions,type_of_interaction,quoted_text_original,quoted_text_original_screenname,retweeting_orig_screenname,orig_text_replied_to,orig_author_replied_to
0,2019-05-03 17:56:09,1124372085114122240,The stakes are high for our community! Fla cou...,<bound method Status.retweets of Status(_api=<...,2,RepDarrenSoto,None,None,None,[ForThePeople],[HouseDemocrats],Dem->Dem,None,None,None,None,None
1,2019-05-03 15:37:27,1124337179633885187,Teachers teach and police protect and serve. A...,<bound method Status.retweets of Status(_api=<...,4,RepDarrenSoto,None,None,None,[],[],,None,None,None,None,None
2,2019-05-03 14:40:51,1124322936809250816,".@HispanicCaucus, @CAPAC, &amp; @OfficialCBC ...",<bound method Status.retweets of Status(_api=<...,17,RepDarrenSoto,None,None,None,[],"[HispanicCaucus, HispanicCaucus, CAPAC, Offici...",,None,None,HispanicCaucus,None,None
3,2019-05-02 21:51:56,1124069034725605377,"U.S. House passes Climate Action Now Act, wou...",<bound method Status.retweets of Status(_api=<...,5,RepDarrenSoto,None,None,None,[],[garywhite13],,None,None,garywhite13,None,None
4,2019-05-02 20:08:20,1124042960537038848,"Thank you for voting in favor of H.R.9, the #...",<bound method Status.retweets of Status(_api=<...,3,RepDarrenSoto,None,None,None,[ClimateActionNow],[NRDCFedGov],,None,None,NRDCFedGov,None,None


### Creating DataFrame of the interacting tweets

In [29]:
interaction_tweets=pd.DataFrame(columns=tweets_dict_data.columns)

In [30]:
sia = SIA()

In [31]:
cntRD=0
cntDR=0
cntRR=0
cntDD=0
user_mention_RD=0
user_mention_DD=0
user_mention_DR=0
user_mention_RR=0
interaction=[]


all_sentiment_score =[]

new_iter = 0
for i_iter in range(len(tweets_dict_data)):
    flag=0
    sentiment_score = sia.polarity_scores(tweets_dict_data.loc[i_iter]['text'])['compound']
    
    # Democrats
    if tweets_dict_data.loc[i_iter].user_screenname in dem_names:
        
        ## D-R
        # retweeted
        if (tweets_dict_data.loc[i_iter].retweeting_orig_screenname in rep_names):
            cntDR+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Rep->Dem')
            
            flag=1
            new_iter+=1  
        #quoted
        elif (tweets_dict_data.loc[i_iter].quoted_text_original_screenname in rep_names):
            cntDR+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Rep->Dem')

            flag=1
            new_iter+=1  
        #post-comment
        elif (tweets_dict_data.loc[i_iter].orig_author_replied_to in rep_names):
            cntDR+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Rep->Dem')
            
            flag=1
            new_iter+=1
        
        ## D-D
        #retweet
        elif (tweets_dict_data.loc[i_iter].retweeting_orig_screenname in dem_names):
            cntDD+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Dem->Dem')
            
            flag=1
            new_iter+=1
        #quoted
        elif (tweets_dict_data.loc[i_iter].quoted_text_original_screenname in dem_names):
            cntDD+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Dem->Dem')

            flag=1
            new_iter+=1
        #post-comment
        elif (tweets_dict_data.loc[i_iter].orig_author_replied_to in dem_names):
            cntDD+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Dem->Dem')
           
            flag=1
            new_iter+=1
            
    #Republican
    elif tweets_dict_data.loc[i_iter].user_screenname in rep_names:
        ## R-D
        #Retweet
        if (tweets_dict_data.loc[i_iter].retweeting_orig_screenname in dem_names):# or  or :
            cntRD+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Dem->Rep')
            
            flag=1
            new_iter+=1
        #quoted
        elif (tweets_dict_data.loc[i_iter].quoted_text_original_screenname in dem_names):
            cntRD+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Dem->Rep')
            
            flag=1
            new_iter+=1
        #post-comment
        elif (tweets_dict_data.loc[i_iter].orig_author_replied_to in dem_names):
            cntRD+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Dem->Rep')
            
            flag=1
            new_iter+=1
            
        ## R-R
        #retweet
        elif (tweets_dict_data.loc[i_iter].retweeting_orig_screenname in rep_names):# or  or :
            cntRR+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Rep->Rep')
            
            flag=1
            new_iter+=1
        #quoted
        elif (tweets_dict_data.loc[i_iter].quoted_text_original_screenname in rep_names):
            cntRR+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Rep->Rep')
            
            flag=1
            new_iter+=1
        #post-comment
        elif (tweets_dict_data.loc[i_iter].orig_author_replied_to in rep_names):
            cntRR+=1
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            interaction.append('Rep->Rep')
            
            flag=1
            new_iter+=1
            
    #user-mentions
    if flag==0:
        if tweets_dict_data.loc[i_iter].type_of_interaction!='':
            if tweets_dict_data.loc[i_iter].type_of_interaction=='Rep->Dem': #R-D
                user_mention_RD+=1
                
            elif tweets_dict_data.loc[i_iter].type_of_interaction=='Dem->Dem': #D-D
                user_mention_DD+=1
                
            elif tweets_dict_data.loc[i_iter].type_of_interaction=='Dem->Rep': #D-R
                user_mention_DR+=1
                
            elif tweets_dict_data.loc[i_iter].type_of_interaction=='Rep->Rep': #R-R
                user_mention_RR+=1
                    
            interaction.append(None)        
            interaction_tweets.loc[new_iter]=tweets_dict_data.loc[i_iter]
            new_iter+=1
        


In [32]:
print("Republican posted and Democrats retweeted/replied/quoted :",cntRD)
print("Democrats posted and Democrats retweeted/replied/quoted :",cntDD)
print("Democrats posted and Republican retweeted/replied/quoted :",cntDR)
print("Republican posted and Republican retweetesd/replied/quoted :",cntRR)
print('\n-Users Mentioned :')
print("Republican posted and Democrats mentioned :",user_mention_RD)
print("Democrats posted and Democrats mentioned :",user_mention_DD)
print("Democrats posted and Republicans mentioned :",user_mention_DR)
print("Republican posted and Republicans mentioned :",user_mention_RR)

Republican posted and Democrats retweeted/replied/quoted : 11
Democrats posted and Democrats retweeted/replied/quoted : 322
Democrats posted and Republican retweeted/replied/quoted : 4
Republican posted and Republican retweetesd/replied/quoted : 147

-Users Mentioned :
Republican posted and Democrats mentioned : 62
Democrats posted and Democrats mentioned : 400
Democrats posted and Republicans mentioned : 25
Republican posted and Republicans mentioned : 363


In [43]:
interaction_tweets['interactions'] = interaction

In [58]:
merged_interaction = []
for i in range(len(interaction_tweets)):
    if interaction_tweets['type_of_interaction'][i]!="":
        merged_interaction.append(interaction_tweets['type_of_interaction'][i])
    else:
        merged_interaction.append(interaction_tweets['interactions'][i])
    

In [59]:
interaction_tweets['final_interactions'] = merged_interaction

##### saving the dataframes into a csv for future use

In [52]:
tweets_dict_data.to_csv('May3_tweets.csv')

In [61]:
interaction_tweets.to_csv('May3_interactions.csv')